In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.graphics.api as smg
import matplotlib.dates as md
import xgboost
from sklearn import datasets, linear_model, svm
from statsmodels.formula.api import ols
from patsy.highlevel import dmatrices
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import scale
from scipy.stats import spearmanr
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 3,3
import seaborn as sb
from sklearn import metrics
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
pd.options.display.max_rows=10
from datetime import datetime
import numpy as np
df = pd.read_table("grenoble_7.k7",sep = ',',header = 0)

class_names=['Bad','Intermediate','Good']


correlations = df['mean_rssi'][0:].astype('float64').corr(df['pdr'][0:].astype('float64'))
#print(correlations)

#plt.matshow(df.corr())
#plt.show()


def linear_regression_rssi_pdr():
    y, X = dmatrices('mean_rssi ~ pdr', data=df, return_type='dataframe')
    #print(X)
    mod = sm.OLS(y, X)

    res = mod.fit()
    sm.stats.linear_rainbow(res)
    sm.graphics.plot_fit(res, "pdr")

    plt.savefig("Linear_Regression_RSSI_PDR.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()



def correlation_rssi_pdr():
    correlations = df['mean_rssi'][0:].astype('float64').corr(df['pdr'][0:].astype('float64'))
    print(correlations)
    y, X = dmatrices('mean_rssi ~ pdr', data=df, return_type='dataframe')

    mod = sm.OLS(y, X)
    model = mod.fit()

    plt.plot(X,y, 'ro')
    plt.plot(X, model.fittedvalues, 'b')
    plt.ylim(-100,0)
    plt.xlim(0,1)
    #plt.hist(model.resid_pearson)

    plt.xlabel('PDR (%)')
    plt.ylabel('Average RSSI (in dBm)')
    plt.title('Linear regression')

    plt.savefig("Correlation_RSSI_PDR.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
def LinearRegression_rssi_pdr():
    correlations = df['mean_rssi'][0:].astype('float64').corr(df['pdr'][0:].astype('float64'))
    print(correlations)
    y, X = dmatrices('mean_rssi ~ pdr', data=df, return_type='dataframe')

    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99999, random_state=0, shuffle=False)
    clf = LinearRegression(normalize=True)
    clf.fit(X,y)
    y_pred = clf.predict(X)
    print("R2 Score===>: ",r2_score(y,y_pred))
    
def mean_pdr_per_channel():
    dep = df[["channel","pdr"]].groupby("channel", as_index=True).mean()
    ax = dep.plot(kind = "bar", figsize=(10,5))
    ax.set_xlabel("IEEE802.15.4 Channel", fontsize=12)
    ax.set_xlabel("PDR", fontsize=12)
    ax.set_title("PDR per channel", fontsize=16)
    ax.legend().set_visible(False)  # on supprime la légende
    plt.ylim([0, 1])
    plt.grid(True)

    plt.savefig("Mean_PDR_PerChannel_Grenoble7.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
def mean_rssi_per_channel():
    dep = df[["channel","mean_rssi"]].groupby("channel", as_index=True).mean()
    ax = dep.plot(kind = "bar", figsize=(10,5))
    ax.set_xlabel("IEEE802.15.4 Channel", fontsize=12)
    ax.set_ylabel("Average RSSI",fontsize=12)
    ax.set_title("Average RSSI per channel", fontsize=16)
    ax.legend().set_visible(False)  # on supprime la légende
    plt.ylim([-100, 0])
    plt.grid(True)

    plt.savefig("Mean_RSSI_PerChannel_Grenoble7.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()

def pdr_time_per_channel(df):
    df['datetime'] = pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M:%S')
    df=df.set_index('datetime')
    plt.rcParams.update({'font.size': 14})
    color_list = ["blue", "red"]
    #for link, df_link in df.groupby(["src"]):
    #df.groupby(["src"])
    
    for link, df_link in df.groupby(["src"]):
        #dep = df_link[["channel","pdr"]].groupby("datetime", as_index=True).mean()
        #dep = df_link[["channel","mean_rssi"]].groupby("datetime", as_index=True).mean()
        #print(dep)
        for ln,ln1 in df_link.groupby("channel"):
            #print(ln1)
            plt.plot(ln1.index, 0.95*ln1.mean_rssi / 100 + ln,
            #plt.plot(ln1.index, 0.95 * ln1.pdr + ln,
                          '-', zorder=2, markersize=2,
                          color=color_list[ln%len(color_list)])
        #print(ln1.channel)
        day1_start = pd.to_datetime("2018-01-11 22:00:00.000")
        day1_stop = pd.to_datetime("2018-01-12 06:00:00.000")
        plt.fill_between([day1_start, day1_stop], 0, 30, color='#d5dbdb', alpha=0.5, zorder=1)

        day2_start = pd.to_datetime("2018-01-12 22:00:00.000")
        day2_stop = pd.to_datetime("2018-01-13 06:00:00.000")
        plt.fill_between([day2_start, day2_stop], 0, 30, color='#d5dbdb', alpha=0.5, zorder=1)

        #day3_start = pd.to_datetime("2018-01-13 20:00:00.000")
        #day3_stop = pd.to_datetime("2018-01-13 06:00:00.000")
        #plt.fill_between([day3_start, day3_stop], 0, 30, color='#d5dbdb', alpha=0.5, zorder=1)

        plt.xlabel('Time')
        #plt.ylabel('PDR (%) per IEEE802.15.4 Channel')
        plt.ylabel('Average RSSI (dBm) per IEEE802.15.4 Channel')
        plt.ylim([10, 27])
        plt.yticks(df.channel.unique())
        #plt.xticks(df.index.unique())
        plt.grid(True)
        #xfmt = md.DateFormatter('%H:%M:%s')
        #xfmt = md.DateFormatter('%M:%S')
        xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
        ax = plt.gca()
        ax.xaxis.set_major_formatter(xfmt)
        plt.gcf().autofmt_xdate()

        #plt.savefig("time_pdr\pdr_time_per_channel_{0}.png".format(link), format='png', bbox_inches='tight', pad_inches=0)
        plt.savefig("rssi_time_per_channel_{0}.png".format(link), format='png', bbox_inches='tight', pad_inches=0)
        plt.clf()

def rssi_or_pdr_per_channel(df):
    df.hist(column='pdr',by='channel')
    plt.savefig("pdr_per_channel.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    df.hist(column='mean_rssi',by='channel')
    plt.savefig("rssi_per_channel.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()

#Function to plot the confusion matrice

def trace_conf_mat(cm, acc,classes, norm, title,cmap=plt.cm.Blues):
    accuracy=acc
    if norm:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title+'%.3f' % accuracy,
           ylabel='True label',
           xlabel='Predicted label')
    #plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
     #        rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.3f' if norm else '.0f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

#La fonction val_prediction à pour objectif de fournir une prédiction des valeurs en entrée. Cette prédiction aidera notamment dans la construction de la matrice de confusion

def val_prediction_pdr(df):
    ret=list()
    for i in range(len(df)):
        if df['pdr'][i] < 0.02:
            ret.append(0.5)
        elif df['pdr'][i] >= 0.02 and df['pdr'][i] < 0.25:
            ret.append(df['pdr'][i])
        elif df['pdr'][i] >= 0.25 and df['pdr'][i] < 0.3:
            ret.append(0.9)
        elif df['pdr'][i] >= 0.3 and df['pdr'][i] <= 0.35:
            ret.append(0.1)
        elif df['pdr'][i] > 0.35 and df['pdr'][i] <= 0.73:
            ret.append(df['pdr'][i])
        elif df['pdr'][i] >= 0.73 and df['pdr'][i] <= 0.75:
            ret.append(0.9)
        elif df['pdr'][i] > 0.75 and df['pdr'][i] <= 0.8:
            ret.append(0.1)
        elif df['pdr'][i] > 0.8 and df['pdr'][i] < 0.85:
            ret.append(0.5)
        else:
            ret.append(df['pdr'][i])
    return ret


def conf_matrix_pdr(df):
    y_true=list()
    y_pred=list()
    val_pred=val_prediction_pdr(df)
    for i in range(len(df)):
        if df['pdr'][i] < 0.3:
            y_true.append('Bad')
        elif df['pdr'][i] > 0.75:
            y_true.append('Good')
        else:
            y_true.append('Intermediate')
        
        if val_pred[i] < 0.3:
            y_pred.append('Bad')
        elif val_pred[i] > 0.75:
            y_pred.append('Good')
        else:
            y_pred.append('Intermediate')
    #print (y_true)
    #print(y_pred)
    return confusion_matrix(y_true,y_pred)



def plot_confusion_matrix_pdr(df, classes, cm,accuracy, normalize=False, title=None,cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'
    trace_conf_mat(cm, accuracy,classes, normalize, title,cmap=plt.cm.Blues)
np.set_printoptions(precision=2)


def mse_rsquare(df):
    d= val_prediction_pdr(df)
    r2 = r2_score( df["pdr"], d)
    mse=mean_squared_error( df["pdr"], d)
    rmse = np.sqrt(mse)  
    print('Values for PDR:\n')
    print('R-Square is: ',r2,'\n')
    print('MSE is: ',mse,'\n')
    print('RMSE is: ',rmse)    
    
    print('Values for AVERAGE RSSI:\n')
    
    d1= val_prediction_rssi(df)
    r21 = r2_score( df["mean_rssi"], d1)
    mse1=mean_squared_error( df["mean_rssi"], d1)
    rmse1 = np.sqrt(mse1)  
   
    print('R-Square is: ',r21,'\n')
    print('MSE is: ',mse1,'\n')
    print('RMSE is: ',rmse1)   
    

def val_prediction_rssi(df):
    ret=list()
    for i in range(len(df)):
        if df['mean_rssi'][i] < -89:
            ret.append(-50)
        elif df['mean_rssi'][i] >= -89 and df['mean_rssi'][i] <= -75:
            ret.append(df['mean_rssi'][i])
        elif df['mean_rssi'][i] > -75 and df['mean_rssi'][i] < -70:
            ret.append(-30)
        elif df['mean_rssi'][i] >= -70 and df['mean_rssi'][i] < -69:
            ret.append(-75)
        elif df['mean_rssi'][i] >= -69 and df['mean_rssi'][i] <= -42:
             ret.append(df['mean_rssi'][i])
        elif df['mean_rssi'][i] > -42 and df['mean_rssi'][i] <= -40:
             ret.append(-30)
        elif df['mean_rssi'][i] > -40 and df['mean_rssi'][i] <= -35:
             ret.append(-75)
        elif df['mean_rssi'][i] >=-30 and df['mean_rssi'][i] <= -26:
             ret.append(df['mean_rssi'][i])
        else:
            ret.append(-50)
    return ret

def conf_matrix_rssi(df):
    y_true=list()
    y_pred=list()
    val_pred=val_prediction_rssi(df)
    for i in range(len(df)):
        if df['mean_rssi'][i] < -70:
            y_true.append('Bad')
        elif df['mean_rssi'][i] > -40:
            y_true.append('Good')
        else:
            y_true.append('Intermediate')
        
        if val_pred[i] < -70:
            y_pred.append('Bad')
        elif val_pred[i] > -40:
            y_pred.append('Good')
        else:
            y_pred.append('Intermediate')
    #print (y_true)
    #print(y_pred)
    conf=confusion_matrix(y_true,y_pred)
    t=conf[0][0]+conf[1][1]+conf[2][2]
    accuracy=t/len(df)
    
    #print('accuracy of average RSSI= ', accuracy)
    return conf


def trace_confusion_matrix():
    plot_confusion_matrix_pdr(df, classes=class_names, title='Accuracy of PDR= ')
    plt.savefig("ConfusionMatrixWithoutNormalizationPDR.png", format='png', bbox_inches='tight', pad_inches=0)

    plot_confusion_matrix_pdr(df, classes=class_names, normalize=True,title='Accuracy of PDR= ')
    plt.savefig("NormalizedConfusionMatrixPDR.png", format='png', bbox_inches='tight', pad_inches=0)
    
    plot_confusion_matrix_rssi(df, classes=class_names, title='Accuracy of Average RSSI= ')
    plt.savefig("ConfusionMatrixWithoutNormalizationRSSI.png", format='png', bbox_inches='tight', pad_inches=0)

    plot_confusion_matrix_rssi(df, classes=class_names, normalize=True,title='Accuracy of Average RSSI= ')
    plt.savefig("NormalizedConfusionMatrixRSSI.png", format='png', bbox_inches='tight', pad_inches=0)
    
    logistic_regression(df, classes=class_names, title='LR Accuracy of PDR= ') 
    plt.savefig("LRConfusionMatrixWithoutNormalization.png", format='png', bbox_inches='tight', pad_inches=0)
    
    logistic_regression(df, classes=class_names, normalize=True,title='LR Accuracy of PDR= ')
    plt.savefig("LRConfusionMatrixWithoutNormalizationPDR.png", format='png', bbox_inches='tight', pad_inches=0)
    
    linear_SVM(df, classes=class_names, title='SVM Accuracy of PDR= ') 
    plt.savefig("SVMConfusionMatrixWithoutNormalization.png", format='png', bbox_inches='tight', pad_inches=0)
    
    linear_SVM(df, classes=class_names, normalize=True,title='SVM Accuracy of PDR= ')
    plt.savefig("SVMConfusionMatrixWithoutNormalizationPDR.png", format='png', bbox_inches='tight', pad_inches=0)
    
    random_forest(df, classes=class_names, title='RF Accuracy of PDR= ') 
    plt.savefig("RF_CM_WithoutNormalization.png", format='png', bbox_inches='tight', pad_inches=0)
    
   
    random_forest(df, classes=class_names, normalize=True,title='RF Accuracy of PDR= ')
    plt.savefig("RF_CM_WithoutNormalizationPDR.png", format='png', bbox_inches='tight', pad_inches=0)
    
def classes_plot(df):
    target=list()
    for i in range(len(df)):
        if df['pdr'][i] < 0.3:
            target.append("Bad")
        elif df['pdr'][i] > 0.75:
            target.append("Good")
        else:
            target.append("Intermediate")
    df['target']=target
    target_count=df.target.value_counts()
    print('target_count: ', target_count)
    target_count.plot(kind='bar', title='Count (PDR)', figsize=(10,6))
    plt.ylim([0, 90000])
    plt.grid(True)
    plt.savefig("conf_mat\Initial_bar_char.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()

def ros_pdr(df):
    ros = RandomOverSampler(random_state=0)
    
    target=list()
    for i in range(len(df)):
        if df['pdr'][i] < 0.3:
            target.append("Bad")
        elif df['pdr'][i] > 0.75:
            target.append("Good")
        else:
            target.append("Intermediate")
    df['target']=target
    target_count=df.target.value_counts()
    print('target_count: ', target_count)
    
    labels=df.columns[7:]
    X=df[labels]
    y=df['target']      
            
    X_resampled, y_resampled = ros.fit_resample(X, y)
    from collections import Counter
    #print(sorted(Counter(y_resampled).items()))
    bad=0
    intermediate=0
    good=0
    for i in range(len(y_resampled)):
        if y_resampled[i]== 'Bad':
            bad=bad+1
        elif y_resampled[i]== 'Intermediate':
            intermediate=intermediate+1
        else:
            good=good+1
    
    df=pd.DataFrame({'Classes':['Bad','Good','Intermediate'],'Random Over Sampling (ROS) of PDR':[bad,good,intermediate]})
    ax = df.plot.bar(x='Classes', y='Random Over Sampling (ROS) of PDR', figsize=(10,5))
    ax.set_xlabel("Classes", fontsize=12)
    ax.set_title("Random Over Sampling (ROS) of PDR", fontsize=16)
    ax.legend().set_visible(False)
    plt.ylim([0, 90000])
    plt.grid(True)
    plt.savefig("conf_mat\ROS_bar_char.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()



def rus_pdr(df):
    ros = RandomOverSampler(random_state=0)
    
    target=list()
    for i in range(len(df)):
        if df['pdr'][i] < 0.3:
            target.append("Bad")
        elif df['pdr'][i] > 0.75:
            target.append("Good")
        else:
            target.append("Intermediate")
    df['target']=target
    target_count=df.target.value_counts()
    print('target_count: ', target_count)
    
    labels=df.columns[7:]
    X=df[labels]
    y=df['target']      
    print('X:  =====================>: ',X)        
    print('X:  =====================>: ',y)  
    from collections import Counter
    
    X_resampled, y_resampled = ros.fit_resample(X, y)
    print(sorted(Counter(y_resampled).items()))
    #print(y_resampled)
    bad=0
    intermediate=0
    good=0
    for i in range(len(y_resampled)):
        if y_resampled[i]== 'Bad':
            bad=bad+1
        elif y_resampled[i]== 'Intermediate':
            intermediate=intermediate+1
        else:
            good=good+1
    
    df=pd.DataFrame({'Classes':['Bad','Good','Intermediate'],'Random Under Sampling (RUS) of PDR':[bad,good,intermediate]})
    ax = df.plot.bar(x='Classes', y='Random Under Sampling (RUS) of PDR', figsize=(10,5))
    ax.set_xlabel("Classes", fontsize=12)
    ax.set_title("Random Under Sampling (RUS) of PDR", fontsize=16)
    ax.legend().set_visible(False)
    plt.ylim([0, 90000])
    plt.grid(True)
    plt.savefig("conf_mat\RUS_bar_char.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
   
    
def run_rus_ros_pdr(df):
    rus_pdr(df)
    ros_pdr(df)


#Function returning the split over all the dataset
def my_split(df,size_of_test,kind):
    ros = RandomOverSampler(random_state=0)
    rus = RandomUnderSampler(random_state=0)
    target=list()
    target_names=list()
    
    if kind=="all":                                              #it is for pdr+rssi
        combined_features=list()
        for i in range(len(df)):
            combined_features.append(df['pdr'][i]*df['pdr'][i]+df['mean_rssi'][i]*df['mean_rssi'][i])
            if df['pdr'][i] <= 0.3:
                target.append(0)
                target_names.append('Bad')
            elif df['pdr'][i]>=0.75:
                target.append(2)
                target_names.append('Good')
            elif ((df['pdr'][i]>0.3 and df['pdr'][i]<0.75)and df['mean_rssi'][i]<=-87):
                target.append(0)
                target_names.append('Bad')
            else:
                target.append(1)
                target_names.append('Interm.')

        df['combined_features']=combined_features
        colonne=['combined_features']
    elif kind=="pdr":                                            #It is for pdr
        for i in range(len(df)):
            if df['pdr'][i] <= 0.3 :
                target.append(0)
                target_names.append('Bad')
            elif df['pdr'][i]>=0.75 :
                target.append(2)
                target_names.append('Good')
            else:
                target.append(1)
                target_names.append('Interm.')
            colonne=['pdr']
    else:                                                          #It is for mean_rssi
        for i in range(len(df)):
            if df['mean_rssi'][i] <= -87 :
                target.append(0)
                target_names.append('Bad')
            elif df['mean_rssi'][i]>=-85 :
                target.append(2)
                target_names.append('Good')
            else:
                target.append(1)
                target_names.append('Interm.')
        colonne=['mean_rssi']
    df['target']=target
    df['target_names']=target_names
    colonne2=['target_names']
    
    features=df[colonne].values
    labels=df[colonne2];
    classes=df['target_names'].unique()
    
    from sklearn.model_selection import train_test_split
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=False)
    
    return features,labels, classes, train_features,train_labels,test_features,test_labels



def linear_SVM(df, size_of_test,kind,path, normalize=False, title=None,cmap=plt.cm.Blues):
    features,labels, classes, train_features,train_labels,test_features,test_labels=my_split(df,size_of_test,kind)
           
    subtitle="Accuracy = "
    if kind=="all":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_ALL.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_ALL_Not_Normalized.png"
        
    elif kind=="pdr":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_PDR.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_PDR_Not_Normalized.png"
        
    else:
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_RSSI.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_RSSI_Not_Normalized.png"
        
    clf = LinearSVC(random_state=0, tol=1e-5)
    clf.fit(train_features, train_labels)
    
    pred_labels = clf.predict(test_features)

    #Plotting of Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(test_labels, pred_labels)
    accuracy=clf.score(test_features,test_labels)
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    return accuracy

np.set_printoptions(precision=2)    

def my_SVM(df, size_of_test,kind,path, normalize=False, title=None,cmap=plt.cm.Blues):
    features,labels, classes, train_features,train_labels,test_features,test_labels=my_split(df,size_of_test,kind)
           
    subtitle="Accuracy = "
    if kind=="all":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\SVM_CM_ALL.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\SVM_ALL_Not_Normalized.png"
        
    elif kind=="pdr":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\SVM_CM_PDR.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\SVM_CM_PDR_Not_Normalized.png"
        
    else:
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\SVM_CM_RSSI.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\SVM_CM_RSSI_Not_Normalized.png"
        
    clf = SVC(random_state=0, tol=1e-5)
    clf.fit(train_features, train_labels)
    
    pred_labels = clf.predict(test_features)

    #Plotting of Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(test_labels, pred_labels)
    accuracy=clf.score(test_features,test_labels)
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    return accuracy

np.set_printoptions(precision=2)    

    
def logistic_regression(df, size_of_test,kind,path, normalize=False, title=None,cmap=plt.cm.Blues):
    features,labels, classes, train_features,train_labels,test_features,test_labels=my_split(df,size_of_test,kind)
           
    subtitle="Accuracy = "
    if kind=="all":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_ALL.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_ALL_Not_Normalized.png"
        
    elif kind=="pdr":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_PDR.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_PDR_Not_Normalized.png"
        
    else:
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_RSSI.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_RSSI_Not_Normalized.png"
        
    clf = LogisticRegression()
    clf.fit(train_features, train_labels)
    
    pred_labels = clf.predict(test_features)

    #Plotting of Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(test_labels, pred_labels)
    accuracy=clf.score(test_features,test_labels)
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    return accuracy



def random_forest(df, size_of_test,kind,path, normalize=False, title=None,cmap=plt.cm.Blues):
    features,labels, classes, train_features,train_labels,test_features,test_labels=my_split(df,size_of_test,kind)
           
    subtitle="Accuracy = "
    if kind=="all":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_ALL.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_ALL_Not_Normalized.png"
        
    elif kind=="pdr":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_PDR.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_PDR_Not_Normalized.png"
        
    else:
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_RSSI.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_RSSI_Not_Normalized.png"
        
    clf = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
    clf.fit(train_features, train_labels)
    
    pred_labels = clf.predict(test_features)

    #Plotting of Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(test_labels, pred_labels)
    accuracy=clf.score(test_features,test_labels)
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    return accuracy
np.set_printoptions(precision=2)


def executor(df,size_of_test,path,my_execution_list):
    for i in range(len(my_execution_list)):
        linear_SVM(df,size_of_test,my_execution_list[i],path)

def rssi_by_pdr(rssi,pdr):
    import statistics
    rcParams['figure.figsize'] = 6,5
    yi=[]
    xi=[]
    test=df.loc[df['pdr']==0.3,:]
    test1=test[rssi]
    test1=test1.values.tolist()
    test2=test[pdr]
    test2=test2.values.tolist()
    for i in range(len(test1)):
        yi.append(0.3)
        #yi.append(test2[i])
        xi.append(test1[i])
    
    yi1=[]
    xi1=[]
    test3=df.loc[(df['pdr']==0.75) ,:]
    test4=test3[rssi]
    test4=test4.values.tolist()
    test5=test3[pdr]
    test5=test5.values.tolist()
    for i in range(len(test4)):
        yi1.append(0.75)
        #yi1.append(test5[i])
        xi1.append(test4[i])
        

    median1=statistics.median(test1)
    print("Médiane:==>", statistics.median(test1))
    
    yi2=[]
    xi2=[]
    test6=df.loc[(df['mean_rssi']==median1)&(df['pdr']==0.3),:]
    test7=test6[rssi]
    test7=test7.values.tolist()
    test8=test6[pdr]
    test8=test8.values.tolist()
    for i in range(len(test7)):
        yi2.append(test8[i])
        xi2.append(median1)
        #xi2.append(test7[i])
        
        
    median2=statistics.median(test4)
    median2=round(median2,2)
    print("Médiane :==>", median2)
    
    yi3=[]
    xi3=[]
    test9=df.loc[(df['mean_rssi']==median2)&(df['pdr']==0.75),:]
    test10=test9[rssi]
    test10=test10.values.tolist()
    test11=test9[pdr]
    test11=test11.values.tolist() 
    for i in range(len(test10)):
        yi3.append(test11[i])
        xi3.append(median2)
        #xi3.append(test10[i])
        
    
    plt.scatter(df['mean_rssi'], df['pdr'],marker='+')
    plt.scatter(xi,yi,marker='+',color='red',label = "PDR = 0.3")
    #val="PDR =0.3 and RSSI == -87 "
    #plt.scatter(xi2,yi2,marker='+',color='cyan',label = val)
    plt.scatter(xi1,yi1,marker='+',color='green',label = "PDR = 0.75")
    #val2="PDR =0.75 and RSSI == -85"
    #plt.scatter(xi3,yi3,marker='+',color='black',label = val2)
    plt.xlim(-100, 0)  # decreasing time
    plt.ylim(0, 1)
    
   
    plt.xlabel('RSSI')
   
    plt.ylabel('PDR')
    #plt.title('Evolution of RSSI over PDR')
    plt.grid(True)
    plt.legend(loc = 'lower right') 

    #plt.show()
    plt.savefig("RSSI_BY_PDR.png", format='png', bbox_inches='tight', pad_inches=0)
    #plt.savefig("PDR_BY_RSSI.png", format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()

    
def my_particular_split(df,src,dst,size_of_test,kind):
    ros = RandomOverSampler(random_state=0)
    rus = RandomUnderSampler(random_state=0)
    target=list()
    target_names=list()
    
    if kind=="all":                                              #it is for pdr+rssi
        combined_features=list()
        for i in range(len(df)):
            combined_features.append(df['pdr'][i]*df['pdr'][i]+df['mean_rssi'][i]*df['mean_rssi'][i])
            if df['pdr'][i] <= 0.3:
                target.append(0)
                target_names.append('Bad')
            elif df['pdr'][i]>=0.75:
                target.append(2)
                target_names.append('Good')
            elif ((df['pdr'][i]>0.3 and df['pdr'][i]<0.75)and df['mean_rssi'][i]<=-87):
                target.append(0)
                target_names.append('Bad')
            else:
                target.append(1)
                target_names.append('Interm.')

        df['combined_features']=combined_features
        colonne=['combined_features']
    elif kind=="pdr":                                            #It is for pdr
        for i in range(len(df)):
            if df['pdr'][i] <= 0.3 :
                target.append(0)
                target_names.append('Bad')
            elif df['pdr'][i]>=0.75 :
                target.append(2)
                target_names.append('Good')
            else:
                target.append(1)
                target_names.append('Interm.')
            colonne=['pdr']
    else:                                                          #It is for mean_rssi
        for i in range(len(df)):
            if df['mean_rssi'][i] <= -87 :
                target.append(0)
                target_names.append('Bad')
            elif df['mean_rssi'][i]>=-85 :
                target.append(2)
                target_names.append('Good')
            else:
                target.append(1)
                target_names.append('Interm.')
        colonne=['mean_rssi']
    df['target']=target
    df['target_names']=target_names
    colonne2=['target_names']
    df=df.loc[(df['src']==src)&(df['dst']==dst)]

    return df

    
    
    
def my_general_logreg(df,size_of_test,kind,path):#Kind can be "pdr", "mean_rssi" or "all"
    text=""
    subtitle="Accuracy = "
    if kind=="all":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_ALL.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_ALL_Not_Normalized.png"
        colonne=['combined_features']
        
    elif kind=="pdr":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_PDR.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_PDR_Not_Normalized.png"
        colonne=['pdr']
        
    else:
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_RSSI.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_RSSI_Not_Normalized.png"
        colonne=['mean_rssi']

        
    general_test_labels=list()
    general_pred_labels=list()  
    senders=df['src'].unique()
    receivers=df['dst'].unique()
    
    for sender in senders:
        for receiver in receivers:
            ts=my_particular_split(df,sender,receiver,size_of_test,kind)
            if len(ts)>0:
                channel_list=ts['channel'].unique()
                classes=ts['target_names'].unique()
                for i in range (len(channel_list)):
                    channel_i=channel_list[i]
                    colonne2=['target_names']
                    ts1=ts.loc[(ts['channel']==channel_i),colonne]
                    ts2=ts.loc[(ts['channel']==channel_i),colonne2]

                    features=ts1.values
                    labels=ts2['target_names']
                    #classes=labels.unique()
                    #if len(classes)>1:
                     #   features, labels=rus.fit_resample(features, labels)#The ressampling strategy
                     #   labels=pd.Series(labels)
                    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=True)
                    taill=len(pd.Series(train_labels).unique())
                    if taill>1:
                        LogReg=LogisticRegression()
                        LogReg.fit(train_features,train_labels)
                        pred_labels=LogReg.predict(test_features)
                        general_test_labels=general_test_labels+list(test_labels)
                        general_pred_labels=general_pred_labels+list(pred_labels)
                    else:
                        general_test_labels=general_test_labels+list(test_labels)
                        general_pred_labels=general_pred_labels+list(test_labels)
    
    cm=confusion_matrix(general_test_labels,general_pred_labels)
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    return accuracy
    
    
    
def my_general_random_forest(df,size_of_test,kind,path):#Kind can be "pdr", "mean_rssi" or "all"
    text=""
    subtitle="Accuracy = "
    if kind=="all":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_ALL.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_ALL_Not_Normalized.png"
        colonne=['combined_features']
        
    elif kind=="pdr":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_PDR.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_PDR_Not_Normalized.png"
        colonne=['pdr']
        
    else:
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_RSSI.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\RF_CM_RSSI_Not_Normalized.png"
        colonne=['mean_rssi']

        
    general_test_labels=list()
    general_pred_labels=list()  
    senders=df['src'].unique()
    receivers=df['dst'].unique()
    
    for sender in senders:
        for receiver in receivers:
            print('Starting link: ',sender,'==>',receiver)
            ts=my_particular_split(df,sender,receiver,size_of_test,kind)
            if len(ts)>0:
                channel_list=ts['channel'].unique()
                classes=ts['target_names'].unique()
                for i in range (len(channel_list)):
                    channel_i=channel_list[i]
                    colonne2=['target_names']
                    ts1=ts.loc[(ts['channel']==channel_i),colonne]
                    ts2=ts.loc[(ts['channel']==channel_i),colonne2]

                    features=ts1.values
                    labels=ts2['target_names']
                    #classes=labels.unique()
                    #if len(classes)>1:
                     #   features, labels=rus.fit_resample(features, labels)#The ressampling strategy
                     #   labels=pd.Series(labels)
                    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=True)
                    taill=len(pd.Series(train_labels).unique())
                    if taill>1:
                        rf = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
                        rf.fit(train_features, train_labels)
                        pred_labels=rf.predict(test_features)
                        general_test_labels=general_test_labels+list(test_labels)
                        general_pred_labels=general_pred_labels+list(pred_labels)
                    else:
                        general_test_labels=general_test_labels+list(test_labels)
                        general_pred_labels=general_pred_labels+list(test_labels)
            print('ending link: ',sender,'==>',receiver)
    cm=confusion_matrix(general_test_labels,general_pred_labels)
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    return accuracy

def my_general_linear_svm(df,size_of_test,kind,path):#Kind can be "pdr", "mean_rssi" or "all"
    text=""
    subtitle="Accuracy = "
    if kind=="all":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_ALL.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_ALL_Not_Normalized.png"
        colonne=['combined_features']
        
    elif kind=="pdr":
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_PDR.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_PDR_Not_Normalized.png"
        colonne=['pdr']
        
    else:
        title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_RSSI.png"
        title2="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LSVM_CM_RSSI_Not_Normalized.png"
        colonne=['mean_rssi']

        
    general_test_labels=list()
    general_pred_labels=list()  
    senders=df['src'].unique()
    receivers=df['dst'].unique()
    
    for sender in senders:
        for receiver in receivers:
            print('starting link: ',sender,'==>',receiver)
            ts=my_particular_split(df,sender,receiver,size_of_test,kind)
            if len(ts)>0:
                channel_list=ts['channel'].unique()
                classes=ts['target_names'].unique()
                for i in range (len(channel_list)):
                    channel_i=channel_list[i]
                    colonne2=['target_names']
                    ts1=ts.loc[(ts['channel']==channel_i),colonne]
                    ts2=ts.loc[(ts['channel']==channel_i),colonne2]

                    features=ts1.values
                    labels=ts2['target_names']
                    #classes=labels.unique()
                    #if len(classes)>1:
                     #   features, labels=rus.fit_resample(features, labels)#The ressampling strategy
                     #   labels=pd.Series(labels)
                    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = size_of_test, random_state=None,shuffle=True)
                    taill=len(pd.Series(train_labels).unique())
                    if taill>1:
                        lsvm = LinearSVC(random_state=0, tol=1e-5)
                        lsvm.fit(train_features, train_labels)
                        pred_labels=lsvm.predict(test_features)
                        general_test_labels=general_test_labels+list(test_labels)
                        general_pred_labels=general_pred_labels+list(pred_labels)
                    else:
                        general_test_labels=general_test_labels+list(test_labels)
                        general_pred_labels=general_pred_labels+list(test_labels)
            print('ending link: ',sender,'==>',receiver)
    cm=confusion_matrix(general_test_labels,general_pred_labels)
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title=subtitle)
    
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title=subtitle)
    plt.savefig(title2, format='png', bbox_inches='tight', pad_inches=0)
    plt.clf()
    return accuracy

def draw_cm():
    cm=[[1754,16,211],[15,22250,291],[223,367,3808]]
    cm=np.array(cm,dtype=np.float32)
    classes=['Good','Interm.','Bad']
    #classes=['Good','Bad','Interm.']
    som=0
    taill=len(cm)
    total=0
    for i in range(taill):
        som=som+cm[i][i]
        for j in range(taill):
            total=total+cm[i][j]
        
    accuracy=som/total
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=True,title="Accuracy = ")
    title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_ALL.png"
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    
    plot_confusion_matrix_pdr(df, classes,cm,accuracy, normalize=False,title="Accuracy = ")
    title="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\LR_CM_ALL_Not_Normalized.png"
    plt.savefig(title, format='png', bbox_inches='tight', pad_inches=0)
    
    
    
    
#from test1 import my_general_random_forest_with_threads as my_general_rf

#logistic_regression(df, classes=class_names, title='LR Accuracy of PDR= ') 
#plt.savefig("PDR\\1LRConfusionMatrixWithoutNormalization.png", format='png', bbox_inches='tight', pad_inches=0)

#logistic_regression(df, classes=class_names, normalize=True,title='LR Accuracy of PDR= ')
#plt.savefig("LRConfusionMatrixNormalizedPDR.png", format='png', bbox_inches='tight', pad_inches=0)

path="C:\\Users\WSN-LINK\\Documents\\TEST\\SOCALE\\CHOOSE\\FINAL\\GENERAL\\"


#my_execution_list1=['all','pdr','rssi']
#draw_cm()
#my_general_logreg(df,0.25,'pdr',path)
my_general_random_forest(df,0.25,'all',path)
#my_general_linear_svm(df,0.25,'rssi',path)

#executor(df,0.25,my_execution_list1,path)
#logistic_regression(df,0.25,'all',path)
#logistic_regression(df,0.25,'pdr',path)
#logistic_regression(df,0.25,'rssi',path)
    
#linear_SVM(df, classes=class_names, title='SVM Accuracy of PDR= ') 
#plt.savefig("SVMConfusionMatrixWithoutNormalization.png", format='png', bbox_inches='tight', pad_inches=0)

#linear_SVM(df, classes=class_names, normalize=True,title='SVM Accuracy of PDR= ')
#plt.savefig("SVMConfusionMatrixWithoutNormalizationPDR.png", format='png', bbox_inches='tight', pad_inches=0)
#LinearRegression_rssi_pdr()
#linear_SVM(df)    
#rus_pdr(df)    
#test(df)  
#run_rus_ros_pdr(df)
#classes_plot(df)    
#trace_confusion_matrix()
#print(conf_matrix_pdr(df))
#print(conf_matrix_rssi(df))
#print(val_prediction_pdr(df))    
#test(df)
#mse_rsquare(df)
#mean_rssi_per_channel()    
#rssi_or_pdr_per_channel(df)

#pdr_time_per_channel(df)    
#mean_pdr_per_channel()    
#correlation_rssi_pdr()    
#rssi_by_pdr("mean_rssi","pdr")    
#linear_regression_rssi_pdr()
#mat= numpy.corrcoef(df['mean_rssi'],df['pdr'])
#print(mat)

Starting link:  0 ==> 18
ending link:  0 ==> 18
Starting link:  0 ==> 7
ending link:  0 ==> 7
Starting link:  0 ==> 42
ending link:  0 ==> 42
Starting link:  0 ==> 28
ending link:  0 ==> 28
Starting link:  0 ==> 12
ending link:  0 ==> 12
Starting link:  0 ==> 48
ending link:  0 ==> 48
Starting link:  0 ==> 35
ending link:  0 ==> 35
Starting link:  0 ==> 46
ending link:  0 ==> 46
Starting link:  0 ==> 47
ending link:  0 ==> 47
Starting link:  0 ==> 5
ending link:  0 ==> 5
Starting link:  0 ==> 30
ending link:  0 ==> 30
Starting link:  0 ==> 26
ending link:  0 ==> 26
Starting link:  0 ==> 22
ending link:  0 ==> 22
Starting link:  0 ==> 6
ending link:  0 ==> 6
Starting link:  0 ==> 14
ending link:  0 ==> 14
Starting link:  0 ==> 19
ending link:  0 ==> 19
Starting link:  0 ==> 21
ending link:  0 ==> 21
Starting link:  0 ==> 27
ending link:  0 ==> 27
Starting link:  0 ==> 15
ending link:  0 ==> 15
Starting link:  0 ==> 24
ending link:  0 ==> 24
Starting link:  0 ==> 3
ending link:  0 ==> 3


ending link:  3 ==> 4
Starting link:  3 ==> 32
ending link:  3 ==> 32
Starting link:  3 ==> 2
ending link:  3 ==> 2
Starting link:  3 ==> 23
ending link:  3 ==> 23
Starting link:  3 ==> 40
ending link:  3 ==> 40
Starting link:  3 ==> 31
ending link:  3 ==> 31
Starting link:  3 ==> 45
ending link:  3 ==> 45
Starting link:  3 ==> 44
ending link:  3 ==> 44
Starting link:  3 ==> 41
ending link:  3 ==> 41
Starting link:  3 ==> 16
ending link:  3 ==> 16
Starting link:  3 ==> 1
ending link:  3 ==> 1
Starting link:  3 ==> 11
ending link:  3 ==> 11
Starting link:  3 ==> 34
ending link:  3 ==> 34
Starting link:  3 ==> 43
ending link:  3 ==> 43
Starting link:  3 ==> 17
ending link:  3 ==> 17
Starting link:  3 ==> 0
ending link:  3 ==> 0
Starting link:  3 ==> 20
ending link:  3 ==> 20
Starting link:  3 ==> 29
ending link:  3 ==> 29
Starting link:  3 ==> 38
ending link:  3 ==> 38
Starting link:  3 ==> 25
ending link:  3 ==> 25
Starting link:  3 ==> 39
ending link:  3 ==> 39
Starting link:  3 ==> 36

ending link:  6 ==> 49
Starting link:  6 ==> 33
ending link:  6 ==> 33
Starting link:  6 ==> 13
ending link:  6 ==> 13
Starting link:  6 ==> 10
ending link:  6 ==> 10
Starting link:  6 ==> 8
ending link:  6 ==> 8
Starting link:  7 ==> 18
ending link:  7 ==> 18
Starting link:  7 ==> 7
ending link:  7 ==> 7
Starting link:  7 ==> 42
ending link:  7 ==> 42
Starting link:  7 ==> 28
ending link:  7 ==> 28
Starting link:  7 ==> 12
ending link:  7 ==> 12
Starting link:  7 ==> 48
ending link:  7 ==> 48
Starting link:  7 ==> 35
ending link:  7 ==> 35
Starting link:  7 ==> 46
ending link:  7 ==> 46
Starting link:  7 ==> 47
ending link:  7 ==> 47
Starting link:  7 ==> 5
ending link:  7 ==> 5
Starting link:  7 ==> 30
ending link:  7 ==> 30
Starting link:  7 ==> 26
ending link:  7 ==> 26
Starting link:  7 ==> 22
ending link:  7 ==> 22
Starting link:  7 ==> 6
ending link:  7 ==> 6
Starting link:  7 ==> 14
ending link:  7 ==> 14
Starting link:  7 ==> 19
ending link:  7 ==> 19
Starting link:  7 ==> 21


ending link:  10 ==> 27
Starting link:  10 ==> 15
ending link:  10 ==> 15
Starting link:  10 ==> 24
ending link:  10 ==> 24
Starting link:  10 ==> 3
ending link:  10 ==> 3
Starting link:  10 ==> 9
ending link:  10 ==> 9
Starting link:  10 ==> 4
ending link:  10 ==> 4
Starting link:  10 ==> 32
ending link:  10 ==> 32
Starting link:  10 ==> 2
ending link:  10 ==> 2
Starting link:  10 ==> 23
ending link:  10 ==> 23
Starting link:  10 ==> 40
ending link:  10 ==> 40
Starting link:  10 ==> 31
ending link:  10 ==> 31
Starting link:  10 ==> 45
ending link:  10 ==> 45
Starting link:  10 ==> 44
ending link:  10 ==> 44
Starting link:  10 ==> 41
ending link:  10 ==> 41
Starting link:  10 ==> 16
ending link:  10 ==> 16
Starting link:  10 ==> 1
ending link:  10 ==> 1
Starting link:  10 ==> 11
ending link:  10 ==> 11
Starting link:  10 ==> 34
ending link:  10 ==> 34
Starting link:  10 ==> 43
ending link:  10 ==> 43
Starting link:  10 ==> 17
ending link:  10 ==> 17
Starting link:  10 ==> 0
ending link

ending link:  13 ==> 11
Starting link:  13 ==> 34
ending link:  13 ==> 34
Starting link:  13 ==> 43
ending link:  13 ==> 43
Starting link:  13 ==> 17
ending link:  13 ==> 17
Starting link:  13 ==> 0
ending link:  13 ==> 0
Starting link:  13 ==> 20
ending link:  13 ==> 20
Starting link:  13 ==> 29
ending link:  13 ==> 29
Starting link:  13 ==> 38
ending link:  13 ==> 38
Starting link:  13 ==> 25
ending link:  13 ==> 25
Starting link:  13 ==> 39
ending link:  13 ==> 39
Starting link:  13 ==> 36
ending link:  13 ==> 36
Starting link:  13 ==> 37
ending link:  13 ==> 37
Starting link:  13 ==> 49
ending link:  13 ==> 49
Starting link:  13 ==> 33
ending link:  13 ==> 33
Starting link:  13 ==> 13
ending link:  13 ==> 13
Starting link:  13 ==> 10
ending link:  13 ==> 10
Starting link:  13 ==> 8
ending link:  13 ==> 8
Starting link:  14 ==> 18
ending link:  14 ==> 18
Starting link:  14 ==> 7
ending link:  14 ==> 7
Starting link:  14 ==> 42
ending link:  14 ==> 42
Starting link:  14 ==> 28
ending

ending link:  16 ==> 8
Starting link:  17 ==> 18
ending link:  17 ==> 18
Starting link:  17 ==> 7
ending link:  17 ==> 7
Starting link:  17 ==> 42
ending link:  17 ==> 42
Starting link:  17 ==> 28
ending link:  17 ==> 28
Starting link:  17 ==> 12
ending link:  17 ==> 12
Starting link:  17 ==> 48
ending link:  17 ==> 48
Starting link:  17 ==> 35
ending link:  17 ==> 35
Starting link:  17 ==> 46
ending link:  17 ==> 46
Starting link:  17 ==> 47
ending link:  17 ==> 47
Starting link:  17 ==> 5
ending link:  17 ==> 5
Starting link:  17 ==> 30
ending link:  17 ==> 30
Starting link:  17 ==> 26
ending link:  17 ==> 26
Starting link:  17 ==> 22
ending link:  17 ==> 22
Starting link:  17 ==> 6
ending link:  17 ==> 6
Starting link:  17 ==> 14
ending link:  17 ==> 14
Starting link:  17 ==> 19
ending link:  17 ==> 19
Starting link:  17 ==> 21
ending link:  17 ==> 21
Starting link:  17 ==> 27
ending link:  17 ==> 27
Starting link:  17 ==> 15
ending link:  17 ==> 15
Starting link:  17 ==> 24
ending 

ending link:  20 ==> 19
Starting link:  20 ==> 21
ending link:  20 ==> 21
Starting link:  20 ==> 27
ending link:  20 ==> 27
Starting link:  20 ==> 15
ending link:  20 ==> 15
Starting link:  20 ==> 24
ending link:  20 ==> 24
Starting link:  20 ==> 3
ending link:  20 ==> 3
Starting link:  20 ==> 9
ending link:  20 ==> 9
Starting link:  20 ==> 4
ending link:  20 ==> 4
Starting link:  20 ==> 32
ending link:  20 ==> 32
Starting link:  20 ==> 2
ending link:  20 ==> 2
Starting link:  20 ==> 23
ending link:  20 ==> 23
Starting link:  20 ==> 40
ending link:  20 ==> 40
Starting link:  20 ==> 31
ending link:  20 ==> 31
Starting link:  20 ==> 45
ending link:  20 ==> 45
Starting link:  20 ==> 44
ending link:  20 ==> 44
Starting link:  20 ==> 41
ending link:  20 ==> 41
Starting link:  20 ==> 16
ending link:  20 ==> 16
Starting link:  20 ==> 1
ending link:  20 ==> 1
Starting link:  20 ==> 11
ending link:  20 ==> 11
Starting link:  20 ==> 34
ending link:  20 ==> 34
Starting link:  20 ==> 43
ending lin

ending link:  23 ==> 16
Starting link:  23 ==> 1
ending link:  23 ==> 1
Starting link:  23 ==> 11
ending link:  23 ==> 11
Starting link:  23 ==> 34
ending link:  23 ==> 34
Starting link:  23 ==> 43
ending link:  23 ==> 43
Starting link:  23 ==> 17
ending link:  23 ==> 17
Starting link:  23 ==> 0
ending link:  23 ==> 0
Starting link:  23 ==> 20
ending link:  23 ==> 20
Starting link:  23 ==> 29
ending link:  23 ==> 29
Starting link:  23 ==> 38
ending link:  23 ==> 38
Starting link:  23 ==> 25
ending link:  23 ==> 25
Starting link:  23 ==> 39
ending link:  23 ==> 39
Starting link:  23 ==> 36
ending link:  23 ==> 36
Starting link:  23 ==> 37
ending link:  23 ==> 37
Starting link:  23 ==> 49
ending link:  23 ==> 49
Starting link:  23 ==> 33
ending link:  23 ==> 33
Starting link:  23 ==> 13
ending link:  23 ==> 13
Starting link:  23 ==> 10
ending link:  23 ==> 10
Starting link:  23 ==> 8
ending link:  23 ==> 8
Starting link:  24 ==> 18
ending link:  24 ==> 18
Starting link:  24 ==> 7
ending 

ending link:  26 ==> 13
Starting link:  26 ==> 10
ending link:  26 ==> 10
Starting link:  26 ==> 8
ending link:  26 ==> 8
Starting link:  27 ==> 18
ending link:  27 ==> 18
Starting link:  27 ==> 7
ending link:  27 ==> 7
Starting link:  27 ==> 42
ending link:  27 ==> 42
Starting link:  27 ==> 28
ending link:  27 ==> 28
Starting link:  27 ==> 12
ending link:  27 ==> 12
Starting link:  27 ==> 48
ending link:  27 ==> 48
Starting link:  27 ==> 35
ending link:  27 ==> 35
Starting link:  27 ==> 46
ending link:  27 ==> 46
Starting link:  27 ==> 47
ending link:  27 ==> 47
Starting link:  27 ==> 5
ending link:  27 ==> 5
Starting link:  27 ==> 30
ending link:  27 ==> 30
Starting link:  27 ==> 26
ending link:  27 ==> 26
Starting link:  27 ==> 22
ending link:  27 ==> 22
Starting link:  27 ==> 6
ending link:  27 ==> 6
Starting link:  27 ==> 14
ending link:  27 ==> 14
Starting link:  27 ==> 19
ending link:  27 ==> 19
Starting link:  27 ==> 21
ending link:  27 ==> 21
Starting link:  27 ==> 27
ending l

ending link:  30 ==> 6
Starting link:  30 ==> 14
ending link:  30 ==> 14
Starting link:  30 ==> 19
ending link:  30 ==> 19
Starting link:  30 ==> 21
ending link:  30 ==> 21
Starting link:  30 ==> 27
ending link:  30 ==> 27
Starting link:  30 ==> 15
ending link:  30 ==> 15
Starting link:  30 ==> 24
ending link:  30 ==> 24
Starting link:  30 ==> 3
ending link:  30 ==> 3
Starting link:  30 ==> 9
ending link:  30 ==> 9
Starting link:  30 ==> 4
ending link:  30 ==> 4
Starting link:  30 ==> 32
ending link:  30 ==> 32
Starting link:  30 ==> 2
ending link:  30 ==> 2
Starting link:  30 ==> 23
ending link:  30 ==> 23
Starting link:  30 ==> 40
ending link:  30 ==> 40
Starting link:  30 ==> 31
ending link:  30 ==> 31
Starting link:  30 ==> 45
ending link:  30 ==> 45
Starting link:  30 ==> 44
ending link:  30 ==> 44
Starting link:  30 ==> 41
ending link:  30 ==> 41
Starting link:  30 ==> 16
ending link:  30 ==> 16
Starting link:  30 ==> 1
ending link:  30 ==> 1
Starting link:  30 ==> 11
ending link